# Classification using neural network (for Iris dataset)
gery, June 30, 2018.
## 1 - Iris dataset description
Iris dataset, from "http://archive.ics.uci.edu/ml/datasets/Iris/", includes 150 examples in total.

Training examples: reshuffled 120 examples

Testing examples: 30 examples, 1st 10 examples with label "1", 2nd 10 examples with label "2" and so on.

## 2 - Structure of the neural network
Input 4 features, output its predicted label, with labels of the form "1", "2" and "3".

In [1]:
import tensorflow as tf
import scipy.io as sio
import numpy as np

In [2]:
data = sio.loadmat('./dataIris.mat')

In [3]:
# training dataset includes reshuffled 120 examples
# test dataset includes 30 examples
# label is 1, 2 and 3
trainData = data['trainDataset']
testData = data['testDataset']

In [4]:
# note that trainData[:,4:5] is different from trainData[:,4]
# print np.shape(trainData[:,4:5])
# print trainData[:,4:5]

In [5]:
learning_rate = 0.1
num_steps = 500
display_step = 100

In [6]:
# Network Parameters
n_hidden_1 = 10 # 1st layer number of neurons
n_hidden_2 = 10 # 2nd layer number of neurons
num_input = 4
num_classes = 1

X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [7]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [8]:
# Create model
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [9]:
# Construct model
logits = neural_net(X)

# Define loss and optimizer
loss_op = tf.norm(logits - Y, 2)
# loss_op = tf.reduce_mean(logits - Y)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.round(logits), Y)
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
prediction = tf.round(logits)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [10]:
# Start training
with tf.Session() as sess:
    
    # Run the initializer
    sess.run(init)

    for step in range(1, num_steps+1):
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: trainData[:,0:4], Y: trainData[:,4:5]})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: trainData[:,0:4],
                                                                 Y: trainData[:,4:5]})
            print("Step " + str(step) + ", Training Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for MNIST test images
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={X: testData[:,0:4],Y: testData[:,4:5]}))
    
    # comparison between true tesing labels and predicted labels
    print "Predicted labels:"
    print sess.run(prediction, feed_dict={X: testData[:,0:4]}).T
    print "True testing labels:"
    print testData[:, 4]

Step 1, Training Loss= 419.2288, Training Accuracy= 0.000
Step 100, Training Loss= 2.4937, Training Accuracy= 0.967
Step 200, Training Loss= 2.4788, Training Accuracy= 0.967
Step 300, Training Loss= 2.4788, Training Accuracy= 0.967
Step 400, Training Loss= 2.4788, Training Accuracy= 0.967
Step 500, Training Loss= 2.4788, Training Accuracy= 0.967
Optimization Finished!
('Testing Accuracy:', 1.0)
Predicted labels:
[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 3. 3. 3. 3.
  3. 3. 3. 3. 3. 3.]]
True testing labels:
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 3. 3. 3. 3.
 3. 3. 3. 3. 3. 3.]
